In [46]:
program = """
foo(1).
foo(2).
bar(X,Y) :- foo(X).
"""

# foo(Y).
# bar(X,Y)?

In [68]:
from pyparsing import Word, alphas, nums, Literal, Or, ParseException
from enum import Enum

fact = Word( alphas ).setResultsName("predicate") + "(" + Word( nums ).setResultsName("value") + ")"
clause = Word( alphas ).setResultsName("predicate") + "(" + Word( alphas ).setResultsName("var") + ")"
rule = Word( alphas ).setResultsName("predicate") + "(" + Word( alphas ).setResultsName("var1") + "," + Word( alphas ).setResultsName("var2") + ")" + ":-" + clause.setResultsName("clause")
# productions = [fact, clause, rule]

class Productions(Enum):
    FACT = 0
    CLAUSE = 1
    RULE = 2
    
parsers = {
    Productions.FACT: fact,
    Productions.CLAUSE: clause,
    Productions.RULE: rule,
}

In [71]:
print(clause.parseString("foo(X)", True))
print(fact.parseString("foo(1)", True))
print(rule.parseString("bar(X,Y) :- foo(X)", True))

['foo', '(', 'X', ')']
['foo', '(', '1', ')']
['bar', '(', 'X', ',', 'Y', ')', ':-', 'foo', '(', 'X', ')']


In [77]:
def parse(program):
    lines = [c.strip() for c in program.split(".")]
    lines = [l for l in lines if l != ""]
    results = list()
    for l in lines:
        for p in Productions:
            parsed = False
            try:
                # print("trying parser", p, "on line", l)
                pp = parsers[p].parseString(l, True)
                if p is Productions.RULE:
                    pp["val1"] = None
                    pp["val2"] = None
                results.append((p, pp))
                parsed = True
                break
            except ParseException as e:
                continue
        if not parsed:
            raise ParseException("All rules failed")
    return results

parse(program)

[(<Productions.FACT: 0>,
  (['foo', '(', '1', ')'], {'predicate': ['foo'], 'value': ['1']})),
 (<Productions.FACT: 0>,
  (['foo', '(', '2', ')'], {'predicate': ['foo'], 'value': ['2']})),
 (<Productions.RULE: 2>,
  (['bar', '(', 'X', ',', 'Y', ')', ':-', 'foo', '(', 'X', ')'], {'predicate': ['bar', 'foo'], 'var1': ['X'], 'var2': ['Y'], 'var': ['X'], 'clause': [(['foo', '(', 'X', ')'], {'predicate': ['foo'], 'var': ['X']})], 'val1': [None], 'val2': [None]}))]

In [94]:
from collections import defaultdict

def evaluate(statements):
    memory = defaultdict(lambda: [])
    while len(statements) > 0:
        p = statements[0]
        new_statements = handle(memory, p[0], p[1])
        statements = statements[1:] + new_statements
#         print(memory, statements)
    return memory
        
def handle(memory, ptype, p):
    new_statements = list()
    if ptype is Productions.FACT:
        pred = p["predicate"]
        value = p["value"]
        memory[pred].append(value)
    if ptype is Productions.RULE:
        print(p["predicate"])
        if p["val1"] is not None and p["val2"] is not None:
            memory[p["predicate"]].append((p["val1"], p["val2"]))
        elif p["clause"] is None and not (p["val1"] is None and p["val2"] is None):
            firstVar = p["val1"] is not None
            for pred, vals in memory.items():
                if isinstance(vals, (list, tuple)):
                    continue
                if firstVar:
                    for v in vals:
                        memory[p["predicate"]].append((p["val1"], v))
                else:
                    for v in vals:
                        memory[p["predicate"]].append((v, p["val2"]))
        else:
            clause = p["clause"]
            cp = clause["predicate"]
            cv = clause["var"]
            p["clause"] = None
            firstVar = cv == p["var1"] and p["val1"] is not None
            for val in memory[cp]:
                if firstVar:
                    p["val1"] = val
                    new_statements.append((Productions.RULE, p))
                else:
                    p["val2"] = val
                    new_statements.append((Productions.RULE, p))
    return new_statements

In [95]:
evaluate(parse(program))

foo
foo
foo


defaultdict(<function __main__.evaluate.<locals>.<lambda>>,
            {'foo': ['1', '2']})

In [ ]:


class Parser(object):
    def __init__(self, program):
        self.program = program
    def separators(self):
        return [".", "?", ]